In [17]:
import tensorflow as tf
import numpy as np
from tdc.single_pred import Tox
from rdkit import Chem

2023-12-27 18:01:21.082933: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-27 18:01:21.260536: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-27 18:01:21.962047: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-11.8/lib64:/usr/local/cuda-11.3/lib64
2023-12-27 18:01:21.962171: W te

In [18]:
def Chem_generator(smiles):
    res_list = []
    for i in tqdm(smiles):
        mol = Chem.MolFromSmiles(i)
        temp = []
        index = 0
        while(len(set(temp))!=4 and index != 100):
            index+=1
            temp.append(Chem.MolToSmiles(mol,doRandom=True))
        res_list+=list(set(temp))
    return res_list

In [19]:
from rdkit.Chem import MolFromSmiles, MolToSmiles
import atomInSmiles
from tqdm import tqdm
import atomInSmiles
def encode(smiles_list,labels):
    mols = []
    res_labels = []
    len_list = []
    
    for smiles in smiles_list:
        mol = MolFromSmiles(smiles)
        mols.append(mol)
    res = []
    for index1,mol in (enumerate(tqdm(mols))):
        index = 0
        res_temp = []
        while(len(res_temp)<20 and index <100):
            random_smiles = MolToSmiles(mol, doRandom=True) # e.g 'C(C(=O)O)N' 
            res_temp.append(random_smiles)
            res_temp = list(set(res_temp))
            index+=1
        res += res_temp
        res_labels += [labels[index1]]*len(res_temp)
        len_list.append(len(res_temp))
        
    # mapping atomID into SMILES srting
    res_train = []


    for smiles in tqdm(res):
        tmp = MolFromSmiles(smiles)
        for atom in tmp.GetAtoms():
            atom.SetAtomMapNum(atom.GetIdx())
        smiles_1 = MolToSmiles(tmp) # 'C([C:1](=[O:2])[OH:3])[NH2:4]' 
        ais_tokens_1 = atomInSmiles.encode(smiles_1, with_atomMap=True).split(' ')
        res_train.append(ais_tokens_1)

    return res_train,res_labels,len_list

In [8]:
Tox_list = ['hERG_Karim','LD50_Zhu','hERG','AMES','DILI','Skin Reaction','Carcinogens_Lagunin','ClinTox']

In [9]:
tox_train,tox_label = Tox(name='herg').get_data(format='DeepPurpose')

Found local copy...
Loading...
Done!


In [36]:
len(encode(tox_train[:5],tox_label[:5])[0])

100%|██████████| 100/100 [00:00<00:00, 458.74it/s]


100

In [16]:
def create_variable(var_name):
    globals()[var_name] = []
def get_variable_value(var_name):
    return globals().get(var_name)


In [38]:
import pickle
for tox in Tox_list:
    create_variable(tox)
    tox_train,tox_label = Tox(name=tox).get_data(format='DeepPurpose')
    temp_train = get_variable_value(tox)
    res = encode(tox_train,tox_label)
    temp_train.append(res)
    with open('./AIS_Tox_data/'+tox,'wb') as file:
        pickle.dump(temp_train,file)


Found local copy...
Loading...
Done!
100%|██████████| 268795/268795 [06:41<00:00, 669.20it/s]
Found local copy...
Loading...
Done!
100%|██████████| 141234/141234 [01:48<00:00, 1297.08it/s]
Found local copy...
Loading...
Done!
[13:46:48] WARNING: not removing hydrogen atom without neighbors
[13:46:48] WARNING: not removing hydrogen atom without neighbors
 86%|████████▋ | 11293/13092 [00:14<00:01, 1198.55it/s][13:47:03] WARNING: not removing hydrogen atom without neighbors
[13:47:03] WARNING: not removing hydrogen atom without neighbors
[13:47:03] WARNING: not removing hydrogen atom without neighbors
[13:47:03] WARNING: not removing hydrogen atom without neighbors
[13:47:03] WARNING: not removing hydrogen atom without neighbors
[13:47:03] WARNING: not removing hydrogen atom without neighbors
[13:47:03] WARNING: not removing hydrogen atom without neighbors
[13:47:03] WARNING: not removing hydrogen atom without neighbors
[13:47:03] WARNING: not removing hydrogen atom without neighbors
[13:

In [27]:
from tdc.utils import retrieve_label_name_list
import pickle
Tox_list_label = ['herg_central','Tox21','ToxCast']
for tox in Tox_list_label:
    label_list = retrieve_label_name_list(tox)
    for label in label_list:

        create_variable(tox+'_'+label)
        tox_train,tox_label = Tox(name=tox,label_name=label).get_data(format='DeepPurpose')
        temp_train = get_variable_value(tox+'_'+label)
        temp_train.append(encode(tox_train,tox_label))
        with open('./AIS_Tox_data/'+tox+'_'+label,'wb') as file:
            pickle.dump(temp_train,file)

Found local copy...
Loading...
Done!
 13%|█▎        | 794780/6137266 [16:12<1:44:04, 855.48it/s] 

In [4]:
from tdc.single_pred import Tox
import pickle
from Module.tokenizer import Tokenizer

Tox_list = ['hERG_Karim','LD50_Zhu','hERG','AMES','DILI','Skin Reaction','Carcinogens_Lagunin','ClinTox']
for name in Tox_list:
    data = Tox(name = name)
    split = data.get_split(method = 'scaffold')
    train,train_label = split['train'].values[:,1],split['train'].values[:,2]
    

Found local copy...
Loading...
Done!
100%|██████████| 13445/13445 [00:05<00:00, 2511.33it/s]
Found local copy...
Loading...
Done!
100%|██████████| 7385/7385 [00:01<00:00, 6285.09it/s]
Found local copy...
Loading...
Done!
100%|██████████| 655/655 [00:00<00:00, 3285.03it/s]
Found local copy...
Loading...
Done!
100%|██████████| 7278/7278 [00:01<00:00, 5238.30it/s]
Found local copy...
Loading...
Done!
100%|██████████| 475/475 [00:00<00:00, 3726.25it/s]
Found local copy...
Loading...
Done!
100%|██████████| 404/404 [00:00<00:00, 6761.71it/s]
Found local copy...
Loading...
Done!
100%|██████████| 280/280 [00:00<00:00, 3294.65it/s]
Found local copy...
Loading...
Done!
100%|██████████| 1478/1478 [00:00<00:00, 3161.20it/s]


In [34]:
import pickle
for tox in Tox_list:
    data = Tox(name = tox)
    split = data.get_split(method = 'scaffold',frac=[0.8,0.2,0])
    train,train_label = split['train'].values[:,1],split['train'].values[:,2]
    val,val_label = split['valid'].values[:,1],split['valid'].values[:,2]
    print(val_label)
    train_res = encode(train,train_label)
    val_res = encode(val,val_label)
    with open(f'./data_sets/{tox}_train','wb') as file:
        pickle.dump(train_res,file)
    with open(f'./data_sets/{tox}_val','wb') as file:
        pickle.dump(val_res,file)

Found local copy...
Loading...
Done!
100%|██████████| 13445/13445 [00:05<00:00, 2466.20it/s]


[1 0 1 ... 1 1 0]


100%|██████████| 53772/53772 [01:23<00:00, 643.57it/s]
Found local copy...
Loading...
Done!
100%|██████████| 7385/7385 [00:01<00:00, 6045.79it/s]


[2.343 2.33 3.207 ... 2.692 1.798 3.0660000000000003]


100%|██████████| 33626/33626 [00:17<00:00, 1912.17it/s]
Found local copy...
Loading...
Done!
100%|██████████| 655/655 [00:00<00:00, 3226.07it/s]


...

100%|██████████| 2248/2248 [00:01<00:00, 1530.98it/s]
Found local copy...
Loading...
Done!
100%|██████████| 280/280 [00:00<00:00, 3229.06it/s]


[0 1 0 0 1 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 1 1 1 0 1 0 0 1 0 0 1 0 0 0 0 0 1 1 0]


100%|██████████| 1120/1120 [00:01<00:00, 824.84it/s]
Found local copy...
Loading...
Done!
100%|██████████| 1478/1478 [00:00<00:00, 3150.57it/s]


[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0]


100%|██████████| 5419/5419 [00:05<00:00, 952.71it/s] 


In [33]:
len(val_label)

296

In [27]:
Tox_list

['hERG_Karim',
 'LD50_Zhu',
 'hERG',
 'AMES',
 'DILI',
 'Skin Reaction',
 'Carcinogens_Lagunin',
 'ClinTox']

In [25]:
split = data.get_split(method = 'scaffold',frac=[0.8,0.2,0])
train,train_label = split['train'].values[:,1],split['train'].values[:,2]
val,val_label = split['train'].values[:,1],split['train'].values[:,2]
tox_train,tox_label = Tox(name=tox).get_data(format='DeepPurpose')

NameError: name 'train_res' is not defined

In [22]:
split['valid']

,Drug_ID,Drug,Y
0,Drug 1324,CCOC(=O)C1(c2ccccc2)CC[NH+](CCc2ccc(N)cc2)CC1,0
1,Drug 100,Nc1cc(N2CCCCC2)nc(N)[n+]1[O-],0
2,Drug 968,Cc1cc(O)c(C(=O)N[C@@H](C(=O)N[C@@H]2C(=O)N3C(C...,0
3,Drug 289,NC(=O)CCC1NC(=O)C(Cc2ccccc2)NC(=O)C(Cc2ccc(O)c...,0
4,Drug 284,NC(=[NH2+])NCCC[C@@H](NC(=O)[C@@H]1CCCN1C(=O)[...,0
...,...,...,...
142,Drug 456,C[C@]12CC(=O)[C@H]3[C@@H](CCC4=CC(=O)C=C[C@@]4...,1
143,Drug 242,Clc1ccccc1C[NH+]1CCc2sccc2C1,0
144,Drug 1427,COC(=O)[C@H](c1ccccc1Cl)[NH+]1CCc2sccc2C1,0
145,Drug 864,CC1(C)[C@H](C(=O)[O-])N2C(=O)C[C@H]2S1(=O)=O,0


In [5]:
with open(f'./data_sets/{name}','rb') as file:
        split = pickle.load(file)
        split['train']

In [16]:
tokenizer = Tokenizer(f'./data_sets/{name}',20)

Load SMILE file ....
Number of SMILEs :  3


  0%|          | 0/3 [00:00<?, ?it/s]


ArgumentError: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(RDKit::ROMol mol, RDKit::SmilesWriteParams params)

In [14]:
split['train'].values[:,1]

array(['CC(C)C[C@H](NC(=O)CNC(=O)c1cc(Cl)ccc1Cl)B(O)O',
       'CCCCCCC[NH+](CC)CCCC(O)c1ccc(NS(C)(=O)=O)cc1',
       'CC[C@@H](c1cccc(O)c1)[C@@H](C)C[NH+](C)C', ...,
       'CCS(=O)(=O)CCn1c([N+](=O)[O-])cnc1C',
       'C=C1/C(=C\\C=C2/CCC[C@@]3(C)[C@H]2CC[C@@H]3[C@H](C)/C=C/[C@@H](O)C2CC2)C[C@@H](O)C[C@@H]1O',
       'CC(C)(C)[NH2+]C[C@H](O)COc1ccccc1C1CCCC1'], dtype=object)

In [ ]:
from tdc.single_pred import Tox
import pickle
from Module.tokenizer import Tokenizer

Tox_list = ['hERG_Karim','LD50_Zhu','hERG','AMES','DILI','Skin Reaction','Carcinogens_Lagunin','ClinTox']
for name in Tox_list:
    data = Tox(name = name)
    split = data.get_split(method = 'scaffold')
    with open(f'./data_sets/{name}','wb') as file:
        pickle.dump(split,file)